In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

import dateutil.parser

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [2]:
import pickle
with open("0130_private_prop_dict.pkl", 'rb') as picklefile: 
    priv_dict = pickle.load(picklefile)
print(priv_dict)

{2411722240007: {'address': '222 9th St S', 'property_id': 2411722240007, 'construction_year': '1922', 'est_market_value': '$169,000', 'owner_name': 'MARY JANE GRAPER', 'building_market_value': '$80,000', 'homestead_status': 'HOMESTEAD', 'tax_market_value': '$146,970', 'total_market_value': '$169,000', 'total_tax': '', 'homestead_mrkt_val_excl': '$22,030', 'total_net_tax': '$2,524.34', 'land_market_value': '$89,000', 'property_type': 'RESIDENTIAL'}, 1602924210184: {'address': '2410 Irving Ave N', 'property_id': '16-029-24-21-0184', 'construction_year': '1968', 'est_market_value': '', 'owner_name': 'BOYS & GIRLS CLUB OF TC', 'building_market_value': '', 'homestead_status': 'NON-HOMESTEAD', 'tax_market_value': '', 'total_market_value': '', 'total_tax': '', 'homestead_mrkt_val_excl': '', 'total_net_tax': '', 'land_market_value': '', 'property_type': 'COMMERCIAL PREFERRED'}, 2202924140554: {'address': '400 1st Ave N', 'property_id': '22-029-24-14-0554', 'construction_year': '1900', 'est_ma

In [3]:
priv_df = pd.DataFrame.from_dict(priv_dict, orient="index")

In [15]:
priv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 696 entries, 102824110123 to 3602924430088
Data columns (total 17 columns):
address                    696 non-null object
property_id                696 non-null object
construction_year          696 non-null object
est_market_value           696 non-null object
owner_name                 696 non-null object
building_market_value      696 non-null object
homestead_status           696 non-null object
tax_market_value           696 non-null object
total_market_value         696 non-null object
total_tax                  696 non-null object
homestead_mrkt_val_excl    696 non-null object
total_net_tax              696 non-null object
land_market_value          696 non-null object
property_type              696 non-null object
sale_date                  269 non-null object
sale_price                 269 non-null object
transaction_type           269 non-null object
dtypes: object(17)
memory usage: 97.9+ KB


In [16]:
priv_df.head()

,address,property_id,construction_year,est_market_value,owner_name,building_market_value,homestead_status,tax_market_value,total_market_value,total_tax,homestead_mrkt_val_excl,total_net_tax,land_market_value,property_type,sale_date,sale_price,transaction_type
102824110123,2730 31St St E,01-028-24-11-0123,1923,,HOLY TRINITY LUTH CH MPLS,,NON-HOMESTEAD,,,,,,,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824210283,3106 Nicollet Ave,03-028-24-21-0283,1989,,METRO TRANSIT COMMISSION,,NON-HOMESTEAD,,,,,,,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824240197,3335 Blaisdell Ave,03-028-24-24-0197,1961,,YOUNG MENS CHRISTIAN ASN T/C,,NON-HOMESTEAD,,,,,,,COMMERCIAL PREFERRED,NaN,NaN,NaN
402824210016,1455 Lake St W,04-028-24-21-0016,1961,"$4,050,000",SONS OF NORWAY,"$3,195,600",NON-HOMESTEAD,"$4,050,000","$4,050,000",,,"$161,031.14","$854,400",COMMERCIAL PREFERRED,NaN,NaN,NaN
502824120005,3033 Excelsior Blvd,05-028-24-12-0005,1955,"$22,820,000",ACKY-CALHOUN LLC,"$16,349,900",NON-HOMESTEAD,"$22,820,000","$22,720,000",,,"$913,934.73","$6,370,100",COMMERCIAL PREFERRED,"August, 2000","$24,500,000",Warranty Deed


In [8]:
#Should I convert nulls to NAs first?


priv_df['building_market_value'].replace('[\$,)]','', regex=True).astype(float) 
priv_df.head()

SyntaxError: invalid syntax (<ipython-input-8-97d22f8c94a1>, line 1)

In [17]:
# Is this an appropriate/best way to deal with blank values?

def replace_empty_values(df):
    df_cl = df.replace(r'\s+', np.nan, regex=True)
    df_cl = df.replace('', np.nan)
    return df_cl

priv_df_cl = replace_empty_values(priv_df)

In [11]:
# First converted empty strings to NaN

priv_df_cl['building_market_value'].replace('[\$,)]','', regex=True).astype(float)
priv_df_cl

,address,property_id,construction_year,est_market_value,owner_name,building_market_value,homestead_status,tax_market_value,total_market_value,total_tax,homestead_mrkt_val_excl,total_net_tax,land_market_value,property_type,sale_date,sale_price,transaction_type
102824110123,2730 31St St E,01-028-24-11-0123,1923,NaN,HOLY TRINITY LUTH CH MPLS,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824210283,3106 Nicollet Ave,03-028-24-21-0283,1989,NaN,METRO TRANSIT COMMISSION,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824240197,3335 Blaisdell Ave,03-028-24-24-0197,1961,NaN,YOUNG MENS CHRISTIAN ASN T/C,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
402824210016,1455 Lake St W,04-028-24-21-0016,1961,"$4,050,000",SONS OF NORWAY,"$3,195,600",NON-HOMESTEAD,"$4,050,000","$4,050,000",NaN,NaN,"$161,031.14","$854,400",COMMERCIAL PREFERRED,NaN,NaN,NaN
502824120005,3033 Excelsior Blvd,05-028-24-12-0005,1955,"$22,820,000",ACKY-CALHOUN LLC,"$16,349,900",NON-HOMESTEAD,"$22,820,000","$22,720,000",NaN,NaN,"$913,934.73","$6,370,100",COMMERCIAL PREFERRED,"August, 2000","$24,500,000",Warranty Deed
502924130012,4021 Thomas Ave N,05-029-24-13-0012,1950,"$180,000",MCA HOLDINGS LLC,NaN,NON-HOMESTEAD,"$180,000","$180,000",NaN,NaN,"$5,765.46","$180,000",LAND COMMERCIAL PREFERRED,NaN,NaN,NaN
511722320050,1101 Lasalle Ave,05-117-22-32-0050,1966,"$367,000",AM PARTNERS LLC,"$166,000",NON-HOMESTEAD,"$367,000","$367,000",NaN,NaN,"$5,756.66","$201,000",RESIDENTIAL TWO UNIT,"April, 2005","$450,000",Warranty Deed
1002824220092,3800 Pleasant Ave,10-028-24-22-0092,1934,NaN,SAGRADO CORAZON DE JESUS,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
1102924140067,910 26Th St E,11-029-24-14-0067,1900,"$149,000",NORDEAST PARTNERS LLC,"$136,800",NON-HOMESTEAD,"$149,000","$149,000",NaN,NaN,"$2,881.82","$12,200",RESIDENTIAL TWO UNIT,"July, 2015","$166,500",Warranty Deed
1102924210015,2701 University SE,11-029-24-21-0015,1946,"$1,995,000",FULLER AVE ESTATES LLC ETAL,"$1,254,900",NON-HOMESTEAD,"$1,995,000","$1,995,000",NaN,NaN,"$78,658.88","$740,100",INDUSTRIAL PREFERRED,NaN,NaN,NaN


In [18]:
priv_df_cl['building_market_value'] = priv_df_cl['building_market_value'].replace('[\$,)]','', regex=True).astype(float)
priv_df_cl

,address,property_id,construction_year,est_market_value,owner_name,building_market_value,homestead_status,tax_market_value,total_market_value,total_tax,homestead_mrkt_val_excl,total_net_tax,land_market_value,property_type,sale_date,sale_price,transaction_type
102824110123,2730 31St St E,01-028-24-11-0123,1923,NaN,HOLY TRINITY LUTH CH MPLS,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824210283,3106 Nicollet Ave,03-028-24-21-0283,1989,NaN,METRO TRANSIT COMMISSION,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
302824240197,3335 Blaisdell Ave,03-028-24-24-0197,1961,NaN,YOUNG MENS CHRISTIAN ASN T/C,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
402824210016,1455 Lake St W,04-028-24-21-0016,1961,"$4,050,000",SONS OF NORWAY,3195600.0,NON-HOMESTEAD,"$4,050,000","$4,050,000",NaN,NaN,"$161,031.14","$854,400",COMMERCIAL PREFERRED,NaN,NaN,NaN
502824120005,3033 Excelsior Blvd,05-028-24-12-0005,1955,"$22,820,000",ACKY-CALHOUN LLC,16349900.0,NON-HOMESTEAD,"$22,820,000","$22,720,000",NaN,NaN,"$913,934.73","$6,370,100",COMMERCIAL PREFERRED,"August, 2000","$24,500,000",Warranty Deed
502924130012,4021 Thomas Ave N,05-029-24-13-0012,1950,"$180,000",MCA HOLDINGS LLC,NaN,NON-HOMESTEAD,"$180,000","$180,000",NaN,NaN,"$5,765.46","$180,000",LAND COMMERCIAL PREFERRED,NaN,NaN,NaN
511722320050,1101 Lasalle Ave,05-117-22-32-0050,1966,"$367,000",AM PARTNERS LLC,166000.0,NON-HOMESTEAD,"$367,000","$367,000",NaN,NaN,"$5,756.66","$201,000",RESIDENTIAL TWO UNIT,"April, 2005","$450,000",Warranty Deed
1002824220092,3800 Pleasant Ave,10-028-24-22-0092,1934,NaN,SAGRADO CORAZON DE JESUS,NaN,NON-HOMESTEAD,NaN,NaN,NaN,NaN,NaN,NaN,COMMERCIAL PREFERRED,NaN,NaN,NaN
1102924140067,910 26Th St E,11-029-24-14-0067,1900,"$149,000",NORDEAST PARTNERS LLC,136800.0,NON-HOMESTEAD,"$149,000","$149,000",NaN,NaN,"$2,881.82","$12,200",RESIDENTIAL TWO UNIT,"July, 2015","$166,500",Warranty Deed
1102924210015,2701 University SE,11-029-24-21-0015,1946,"$1,995,000",FULLER AVE ESTATES LLC ETAL,1254900.0,NON-HOMESTEAD,"$1,995,000","$1,995,000",NaN,NaN,"$78,658.88","$740,100",INDUSTRIAL PREFERRED,NaN,NaN,NaN


In [36]:
def clean_scrape_data(df):
    df['building_market_value'].replace('[\$,)]','', regex=True).astype(float)
    df['homestead_mrkt_val_excl'].replace('[\$,)]','', regex=True).astype(float)
    df['total_market_value'].replace('[\$,)]','', regex=True).astype(float)
    df['total_tax'].replace('[\$,)]','', regex=True).astype(float)
    df['est_market_value'].replace('[\$,)]','', regex=True).astype(float)
    df['land_market_value'].replace('[\$,)]','', regex=True).astype(float)
    df['sale_price'].replace('[\$,)]','', regex=True).astype(float)

    # priv_df_cl['construction_year'].astype(int)

    # parse_time = lambda x: dateutil.parser.parse(x, dayfirst=False)
    # priv_df_cl['sale_date_object'] = priv_df['sale_date'].apply(parse_time)

clean_scrape_data(priv_df_cl)
    
priv_df_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 696 entries, 102824110123 to 3602924430088
Data columns (total 17 columns):
building_market_value      610 non-null object
homestead_mrkt_val_excl    149 non-null object
property_type              696 non-null object
total_market_value         628 non-null object
owner_name                 696 non-null object
total_tax                  0 non-null float64
est_market_value           628 non-null object
homestead_status           696 non-null object
tax_market_value           628 non-null object
total_net_tax              628 non-null object
property_id                696 non-null object
construction_year          664 non-null object
address                    696 non-null object
land_market_value          628 non-null object
sale_price                 269 non-null object
sale_date                  269 non-null object
transaction_type           244 non-null object
dtypes: float64(1), object(16)
memory usage: 97.9+ KB


In [40]:
priv_df.head()

,building_market_value,homestead_mrkt_val_excl,property_type,total_market_value,owner_name,total_tax,est_market_value,homestead_status,tax_market_value,total_net_tax,property_id,construction_year,address,land_market_value,sale_price,sale_date,transaction_type
102824110123,,,COMMERCIAL PREFERRED,,HOLY TRINITY LUTH CH MPLS,,,NON-HOMESTEAD,,,01-028-24-11-0123,1923,2730 31St St E,,NaN,NaN,NaN
302824210283,,,COMMERCIAL PREFERRED,,METRO TRANSIT COMMISSION,,,NON-HOMESTEAD,,,03-028-24-21-0283,1989,3106 Nicollet Ave,,NaN,NaN,NaN
302824240197,,,COMMERCIAL PREFERRED,,YOUNG MENS CHRISTIAN ASN T/C,,,NON-HOMESTEAD,,,03-028-24-24-0197,1961,3335 Blaisdell Ave,,NaN,NaN,NaN
402824210016,"$3,195,600",,COMMERCIAL PREFERRED,"$4,050,000",SONS OF NORWAY,,"$4,050,000",NON-HOMESTEAD,"$4,050,000","$161,031.14",04-028-24-21-0016,1961,1455 Lake St W,"$854,400",NaN,NaN,NaN
502824120005,"$16,349,900",,COMMERCIAL PREFERRED,"$22,720,000",ACKY-CALHOUN LLC,,"$22,820,000",NON-HOMESTEAD,"$22,820,000","$913,934.73",05-028-24-12-0005,1955,3033 Excelsior Blvd,"$6,370,100","$24,500,000","August, 2000",Warranty Deed


In [ ]:
priv_df_cl.group_by("Address").

In [37]:
df = pd.read_csv('../data/raw/wcmsp-185935.csv')

In [38]:
cols = ['address', 'postal_code', 'energy_star_score', 'primary_property_type',
       'building_floor_area', 'parking_floor_area',
       'year_built', 'total_ghg_emissions',
       'site_eui', 'weather_normalized_site_eui',
       'source_eui', 'weather_normalized_source_eui',
       'water_use', 'unnamed']
df.columns = cols
df.head()

,address,postal_code,energy_star_score,primary_property_type,building_floor_area,parking_floor_area,year_built,total_ghg_emissions,site_eui,weather_normalized_site_eui,source_eui,weather_normalized_source_eui,water_use,unnamed
0,201 Chicago Ave S,55415,36,Office,62400,0,2012,971,115,118,251,252,708,NaN
1,1011 Nicollet Mall,55403,68,Office,59479,0,1920,986,90,93,256,266,882,NaN
2,4701 Beard Ave S,55410,0,Other,59172,0,1914,1622,157,160,427,428,163,NaN
3,8 60Th St W,55419,76,Worship Facility,60640,62000,1959,268,50,54,77,81,674,NaN
4,700 10Th Ave S,55415,89,Office,60528,0,1920,504,74,77,136,140,414,NaN


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 14 columns):
address                          257 non-null object
postal_code                      257 non-null object
energy_star_score                257 non-null int64
primary_property_type            257 non-null object
building_floor_area              257 non-null int64
parking_floor_area               257 non-null int64
year_built                       257 non-null int64
total_ghg_emissions              257 non-null int64
site_eui                         257 non-null int64
weather_normalized_site_eui      257 non-null int64
source_eui                       257 non-null int64
weather_normalized_source_eui    257 non-null int64
water_use                        257 non-null int64
unnamed                          0 non-null float64
dtypes: float64(1), int64(10), object(3)
memory usage: 28.2+ KB
